In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/workspaces/codespaces-jupyter/data/processed/harga_sayuran_clean.csv' 
data = pd.read_csv(file_path)


In [ ]:
# Convert 'Tanggal Harga' to datetime, handle inconsistent date formats
data['Tanggal Harga'] = pd.to_datetime(data['Tanggal Harga'], dayfirst=True, errors='coerce')


In [ ]:
# Drop rows with invalid dates
data = data.dropna(subset=['Tanggal Harga'])

In [ ]:
# Extract features from the date
data['Year'] = data['Tanggal Harga'].dt.year
data['Month'] = data['Tanggal Harga'].dt.month

In [ ]:
# Select features and target variable
features = ['Year', 'Month', 'Jenis Sayuran', 'Season']
target = 'Harga Beli Pasar per Kilogram'


In [ ]:
# One-hot encode categorical variables
data_encoded = pd.get_dummies(data[features])

In [ ]:
# Combine with target variable
data_encoded[target] = data[target]

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data_encoded, test_size=0.2, random_state=42)


In [ ]:
# Separate features and target variable
X_train = train_data.drop(columns=[target])
y_train = train_data[target]
X_test = test_data.drop(columns=[target])
y_test = test_data[target]

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Initialize the SVR model
svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)


In [ ]:
# Train the model
svr.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svr.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
# Create future dates for the next 3-6 months
future_dates = pd.date_range(start='2024-07-01', periods=6, freq='M')

# Generate the feature set for future dates
future_data = pd.DataFrame({
    'Year': future_dates.year,
    'Month': future_dates.month,
    'Jenis Sayuran': 'Sawi Putih',  # Example for one vegetable, you can loop for others
    'Season': 'Kemarau'  # Assuming the next few months are dry season
})

In [ ]:
# One-hot encode categorical variables
future_data_encoded = pd.get_dummies(future_data)
future_data_encoded = future_data_encoded.reindex(columns=X_train.columns, fill_value=0)

# Standardize the future data
future_data_scaled = scaler.transform(future_data_encoded)

# Make future predictions
future_predictions = svr.predict(future_data_scaled)


In [ ]:
# Prepare the results for the dashboard
forecast_results = pd.DataFrame({
    'Date': future_dates,
    'Predicted Price per Kilogram': future_predictions
})

print(forecast_results)


In [ ]:
# Save the trained model
import joblib
joblib.dump(svr, 'svr_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Save the forecast results
forecast_results.to_csv('forecast_results.csv', index=False)